Note: 
- previously I using SISS to perform ETL and found a lack of configurational issues, so I decided to use Python instead of SQL Server Integrate System
- link to gsheet [link to gsheet](https://docs.google.com/spreadsheets/d/1js7cpHdC-CIgTs-GqAXKkpOlasi_7i-d38Bfx9A-U9o/edit?usp=sharing)

In [2]:
# pip install pandas mysql-connector-python gspread oauth2client

In [3]:
#import package to jupiter notebook
import numpy as np
import pandas as pd
import json
import re
import time
from urllib.parse import quote
import sqlalchemy as sql
from sqlalchemy.pool import NullPool
# import gc
from datetime import datetime, timedelta
from sqlalchemy.sql import text
# import calendar
# import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread

pd.set_option('display.max_columns', None)
from playsound import playsound

# playsound('avenger.mpeg')

In [20]:
# credential to acces database
username = "root"
password = "root"
host = "127.0.0.1"

connect_string_local = 'mysql://'+username+':'+password+'@'+host+'/?charset=utf8mb4'
SQL_ENGINE_LOCAL = sql.create_engine(connect_string_local, poolclass=NullPool)
SQL_ENGINE_LOCAL.connect()

In [9]:
# get data from gsheet

# Replace the path with the actual path to your downloaded JSON file
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/GDA-User/Documents/Reyki/kerja/2023 by project/Data Science Project/amman/ferrous-destiny-354809-46b226857da6.json')
gc = gspread.authorize(credentials)

# Replace 'Your Google Sheets Name' with the name of your Google Sheets document
spreadsheet = gc.open('Training History')

# Replace 'Sheet1' with the name of the sheet you want to read
worksheet = spreadsheet.get_worksheet(0)  # Assuming you want the first sheet

# Get all values from the worksheet
data = worksheet.get_all_values()

# Create a DataFrame
columns = data[0]  # Assuming the first row contains column names
df_sheet = pd.DataFrame(data[1:], columns=columns)

# Print the DataFrame
print(df_sheet)



  Id  PosID                PosTitle EmployeeID StartDate     EndDate
0  1  50000              IT Manager   10105001  1/1/2022   2/28/2022
1  2  50001          IT Sr. Manager   10105001  3/1/2022  12/31/2022
2  3  50002      Programmer Analyst   10105002  1/1/2022   2/28/2022
3  4  50003  Sr. Programmer Analyst   10105002  3/1/2022  12/31/2022
4  5  50004                IT Admin   10105003  1/1/2022   2/28/2022
5  6  50005            IT Secretary   10105003  3/1/2022  12/31/2022


In [21]:
# get data from database
q="""SELECT * from attn.employee
"""
df_sql = pd.read_sql_query(q, SQL_ENGINE_LOCAL)
df_sql

,Id,EmployeeID,FullName,BirthDate,Address
0,1,10105001,Ali Anton,1982-08-19,Jakarta Utara
1,2,10105002,Rara Siva,1982-01-01,Mandalika
2,3,10105003,Rini Aini,1982-02-20,Sumbawa Besar
3,4,10105004,Budi,1982-02-22,Mataram Kota


In [24]:
# Perform a left join on 'EmployeeID'
result_df = pd.merge( df_sql,df_sheet, on='EmployeeID', how='left')
result_df

,Id_x,EmployeeID,FullName,BirthDate,Address,Id_y,PosID,PosTitle,StartDate,EndDate
0,1,10105001,Ali Anton,1982-08-19,Jakarta Utara,1,50000,IT Manager,1/1/2022,2/28/2022
1,1,10105001,Ali Anton,1982-08-19,Jakarta Utara,2,50001,IT Sr. Manager,3/1/2022,12/31/2022
2,2,10105002,Rara Siva,1982-01-01,Mandalika,3,50002,Programmer Analyst,1/1/2022,2/28/2022
3,2,10105002,Rara Siva,1982-01-01,Mandalika,4,50003,Sr. Programmer Analyst,3/1/2022,12/31/2022
4,3,10105003,Rini Aini,1982-02-20,Sumbawa Besar,5,50004,IT Admin,1/1/2022,2/28/2022
5,3,10105003,Rini Aini,1982-02-20,Sumbawa Besar,6,50005,IT Secretary,3/1/2022,12/31/2022
6,4,10105004,Budi,1982-02-22,Mataram Kota,NaN,NaN,NaN,NaN,NaN


In [ ]:
result_df

In [25]:

# Writing the DataFrame to the SQL database
result_df.to_sql(con=SQL_ENGINE_LOCAL,
                 name='result_amman',
                 schema='attn',
                 index=False,
                 if_exists='append',  # Change to 'replace' if you want to replace the table
                 method='multi',  # Use 'multi' for improved performance
                 chunksize=500  # Adjust the chunk size based on your available memory
                 )

7

In [27]:
#save into csv
result_df.to_csv("result.csv")